In [2]:
import torch, gym, time
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

from src.CNN import CNN

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def play_game(black_player=None, white_player=None, max_moves=300):
    go_env = gym.make('gym_go:go-v0', size=5, komi=0, reward_method='heuristic')
    go_env.reset()

    if black_player and white_player:
        go_env.step(go_env.uniform_random_action())
        go_env.step(go_env.uniform_random_action())

    for _ in range(max_moves):
        # Player 1's turn
        if go_env.done: break
        if black_player: 
            moves = black_player.forward(go_env.state()).detach().cpu().numpy() * go_env.valid_moves()
            go_env.step(moves.argmax())
        else: go_env.step(go_env.uniform_random_action())

        # Player 2's turn
        if go_env.done: break
        if white_player: 
            moves = white_player.forward(go_env.state()).detach().cpu().numpy() * go_env.valid_moves()
            go_env.step(moves.argmax())
        else: go_env.step(go_env.uniform_random_action())
    return go_env.reward()

In [5]:
def get_win_percent(black_player=None, white_player=None, n_games=1000, max_moves=300):
    black, white, draws = 0, 0, 0
    # results = Parallel(n_jobs=6)(delayed(play_game)(black_player, white_player, max_moves) for _ in range(n_games))
    for _ in range(n_games):
        res = play_game(black_player, white_player, max_moves)
        if res > 0: black += 1
        elif res < 0: white += 1
        else: draws += 1
    return (black/n_games)*100, (white/n_games)*100, (draws/n_games)*100

In [6]:
models = []
for i in range(5):
    model = CNN()
    model.to(device)
    model.load_state_dict(torch.load(f"src/models/1000-games/{i}-times.pth"))
    models.append(model)

In [7]:
results = Parallel(n_jobs=6)(delayed(get_win_percent)(b, w) for b in models for w in models)

Model 0:


c:\Users\erik\miniconda3\envs\ml2\lib\site-packages\gym\utils\passive_env_checker.py:21: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  f"It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: {observation_space.dtype}. "
c:\Users\erik\miniconda3\envs\ml2\lib\site-packages\gym\utils\passive_env_checker.py:26: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  "It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. "
c:\Users\erik\miniconda3\envs\ml2\lib\site-packages\gym\core.py:318: DeprecationWarning: WARN: Initializing wrapp

  0: b=37.72%, w=61.08%, d=1.20%, took 22.61 seconds
  1: b=35.33%, w=62.87%, d=1.80%, took 41.77 seconds
  2: b=43.71%, w=53.89%, d=2.40%, took 32.34 seconds
  3: b=35.33%, w=61.68%, d=2.99%, took 34.89 seconds
  4: b=41.92%, w=56.89%, d=1.20%, took 21.36 seconds
Model 1:
  0: b=38.92%, w=61.08%, d=0.00%, took 22.72 seconds
  1: b=43.11%, w=53.89%, d=2.99%, took 34.47 seconds
  2: b=29.34%, w=68.86%, d=1.80%, took 30.03 seconds
  3: b=43.71%, w=55.69%, d=0.60%, took 27.97 seconds
  4: b=37.72%, w=60.48%, d=1.80%, took 24.53 seconds
Model 2:
  0: b=40.12%, w=59.28%, d=0.60%, took 23.28 seconds
  1: b=41.32%, w=57.49%, d=1.20%, took 23.68 seconds
  2: b=40.72%, w=59.28%, d=0.00%, took 22.84 seconds
  3: b=44.91%, w=53.89%, d=1.20%, took 29.65 seconds
  4: b=40.12%, w=58.08%, d=1.80%, took 30.87 seconds
Model 3:
  0: b=36.53%, w=61.68%, d=1.80%, took 29.31 seconds
  1: b=36.53%, w=61.08%, d=2.40%, took 32.99 seconds
  2: b=40.72%, w=58.68%, d=0.60%, took 25.10 seconds
  3: b=37.72%, w=62

In [8]:
# Using Numpy to create an array X
loss_values = np.arange(0, math.pi*2, 0.05)
# Assign variables to the y axis part of the curve
y = np.sin(X)
z = np.cos(X)
# Plotting both the curves simultaneously
plt.plot(X, y, color='r', label='sin')
plt.plot(X, z, color='g', label='cos')
# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Angle")
plt.ylabel("Magnitude")
plt.title("Sine and Cosine functions")
# Adding legend, which helps us recognize the curve according to it's color
plt.legend()
# To load the display window
plt.show()

(35.32934131736527, 61.67664670658682, 2.9940119760479043)
